In [228]:
import numpy as np
import pandas as pd
import warnings
import json
import requests
import random
from elasticsearch import Elasticsearch
warnings.filterwarnings('ignore')

# Exploračná analýza

## Tabuľka events

In [2]:
events = pd.read_csv('vi_data/events_train.csv')
events.sample(10)

,customer_id,timestamp,event_type,product_id,title,category_name,price
12934742,82b3c205-ac14-4268-a11b-e1a24fa65730,2019-08-26 04:31:42.512728 UTC,view_item,a209c9e3-59c6-4567-8848-4b550ca5f8a5,Trendyol Yellow Ruffle Detail Dress,Women>Women's Clothing>Dresses>Mini dress,23.11
4391614,3d998ca6-1482-4164-9990-e59ca595fae8,2019-07-22 08:08:47.143756 UTC,view_item,fcd526e8-da12-46d3-8fd4-e8ae1acc3476,Pierre Cardin Tipped Polo Shirt Mens,Men>Men Clothing>T-Shirts & Polo shirts>Polo S...,21.86
6520833,aa11ddba-81f7-4d97-ae70-002fd8304fd9,2019-07-31 21:06:09.270055 UTC,view_item,8c00cff6-c1d5-4d9d-8359-2101884a0c49,LA Gear Woven Shorts Ladies,Women's Clothing>Shorts>Women>Sports shorts,9.19
10056638,29e666dd-22a6-4813-a35c-b9871d8e207f,2019-08-14 14:07:31.739937 UTC,view_item,a447063b-112e-4de6-a338-26b5bb526193,Nike Air Max Ivo Junior Boys,Children>Children's Footwear>Sneakers>Low snea...,89.81
5866694,9b689a93-5256-475c-a5ff-bf5bab243bdb,2019-07-29 10:04:45.618652 UTC,view_item,6a393972-b6d4-4039-ac94-acafdbb95307,Muddyfox Sunglasses 100 Adults,Men>Men Accessories>Sunglasses,14.91
5446731,8ea10a87-9128-4fe4-8f33-d9e3ee33b472,2019-07-27 14:12:50.712225 UTC,view_item,e16d9ce5-5d00-4865-8eab-c4346363c2e2,Trendyol Purple Lace Detail Dress,Women>Women's Clothing>Dresses,43.49
489295,caeaf557-3c2b-480a-86c7-0604086641bd,2019-07-03 08:53:36.87434 UTC,view_item,83b1d8b7-f90c-4d7a-808c-319c0522d01c,Adidas Crazychaos Mens Trainers,NaN,88.12
782948,993ff133-d5f0-4fbd-9e83-35a28a686d71,2019-07-04 17:13:16.005438 UTC,view_item,ad3cbbad-db3d-46ae-bf9e-2f119ca0ba1a,Lee Cooper Cooper Essentials Crew Neck T Shirt...,NaN,13.68
2483776,b37482b8-31eb-4bc5-be73-c240972c6b6e,2019-07-13 07:47:01.871323 UTC,view_item,de08d80a-59f9-41de-8be2-8aec341326de,Lee Cooper Cooper Basic Vest Mens,NaN,6.66
14337944,7479e25c-143c-4ec4-99d3-964583271ac9,2019-08-30 22:03:20.109117 UTC,view_item,ceb3ace6-9874-4320-9485-5eebdda0f2a8,Trendyol Purple Belt Detailed Knitting Dress,Women>Women's Clothing>Dresses>Summer and beac...,11.99


In [4]:
events.shape

(14614385, 7)

In [5]:
events.event_type.unique()

array(['view_item'], dtype=object)

## Tabuľka purchases

In [3]:
purchases = pd.read_csv('vi_data/purchases_train.csv')
purchases.sample(10) 

,customer_id,timestamp,event_type,product_id,title,category_name,price
97576,f103cb77-74a1-4321-9c95-0080b876dce1,2019-08-04 14:20:01.114696 UTC,purchase_item,b572f0d3-5d63-4695-808f-932969ba1bdb,Kids socks MANY MORNINGS,Children>Girl's Clothing>Underwear>Socks,3.51
97947,39843a42-b9f2-4858-b000-b960fa9ec739,2019-08-04 16:22:00.629292 UTC,purchase_item,6e82ee55-6a61-48f0-ae26-bf73ebce4d86,Womens hoodie Fruit of the Loom Basic,Women>Women's Clothing>Sweatshirts>Hoodies,9.01
65238,1ae583a2-5730-4ff6-afa6-d76ee7757b7a,2019-07-25 11:27:55.507235 UTC,purchase_item,023ed07b-b1f4-4b75-be0d-56330528a799,Muddyfox Cycle Jacket Mens,Men>Men Clothing>Jackets & Coats>Lightweight s...,26.49
54762,a3c411c0-f31a-4f4d-9140-f1a766ae3b46,2019-07-20 20:24:40.817386 UTC,purchase_item,c0bc3023-a401-4790-ad6c-7e58ff8fb975,Trendyol Orange high waist Bikini bottom,Women>Women's Clothing>Swimmsuits & bikins,11.58
124888,6c6fb741-1d37-4570-ae63-5e758875a5d2,2019-08-12 09:38:44.866307 UTC,purchase_item,73608106-72cd-4370-b6d4-a3bfd599b7c0,Nike Tanjun Girls Trainers,Women>Women's Footwear>Sneakers>Sport sneakers,52.43
75922,49e43e9f-a8b0-44bc-b5c8-3e70df3cfb4d,2019-07-29 08:31:00.299825 UTC,purchase_item,d94e8a8d-64e0-4fca-953f-9b762f49a7db,Women's Dress NUMOCO 59,Women>Women's Clothing>Dresses>Mini dress,29.40
90685,19dd5075-3d49-4a87-9011-a0a51fb58a61,2019-08-02 17:14:04.209189 UTC,purchase_item,c29fe27a-2146-4d77-8163-b0f6294e0354,Trendyol Khaki Swimsuit,Women>Women's Clothing>Swimmsuits & bikins>One...,28.56
178054,8d67f4be-60e5-44a8-a225-0c29e1c4d284,2019-08-28 19:27:07.030909 UTC,purchase_item,b8e230bc-dbb1-492b-9799-9eb9582c40ba,Slazenger 3 Pack Briefs Mens,Men>Men Clothing>Underwear>Briefs,9.96
126916,1809e145-1096-456a-b0b0-2fc99d3438b9,2019-08-12 18:55:47.886959 UTC,purchase_item,a2e6be59-8f01-4543-b536-09403038d415,Women's Briefs Lee Cooper 2 Pack,Women>Women's Clothing>Underwear>Panties,4.41
120498,6b836d2b-3c46-4db7-8c38-d7550330b57c,2019-08-10 22:07:49.137156 UTC,purchase_item,3973d445-ba2a-4eb4-8bc1-b227a1faa9ea,Lee Cooper Boxers 5 Pack,Men>Men Clothing>Underwear,11.52


In [7]:
purchases.shape

(188712, 7)

In [8]:
purchases.event_type.unique()

array(['purchase_item'], dtype=object)

tabulky events a purchaches majú roznaké stĺpce 

## Spojenie events and purchases

In [4]:
data = pd.concat([events, purchases], ignore_index=True)
data.sample(10)

,customer_id,timestamp,event_type,product_id,title,category_name,price
7432605,6abcc5fa-ad10-48e2-a5a5-443d98e32138,2019-08-04 14:53:48.692227 UTC,view_item,2d3d0e2c-112f-468f-878b-694dfbcf501b,Firetrap LDN Bum Bag 93,Accessories >Backpacks and Bags>Bum bags,20.45
9645703,8d8feb23-b700-47e6-8509-34a3f8413a10,2019-08-12 20:01:36.577007 UTC,view_item,15b3e401-6159-46e2-94dd-1456cd144597,Puma Court Breaker Mens Trainers,Men>Men Footwear>Sneakers>Low sneakers,50.62
12831915,1ceedea3-9d2f-41ea-ab06-410e157ce2bb,2019-08-25 17:17:28.629495 UTC,view_item,7093f50c-c7d9-4015-ab6e-bc832b9d44ff,Lemoniade Woman's Sweater LS213,Women>Women's Clothing>Sweaters & Cardigans>Ca...,34.12
2380034,69080ffe-407a-44ee-91bc-e82d4e9d6e5d,2019-07-12 16:14:09.839359 UTC,view_item,a1547fd5-c6c0-4dc0-989d-5fe286c893d6,DC Blitz II Trainers Mens,NaN,73.69
7734169,817d8add-1e27-4db9-94d2-6624e89e0e15,2019-08-05 15:48:34.34332 UTC,view_item,827bda66-2718-4549-8d56-39bfd1772a5b,Pierre Cardin Stripe Swimshorts Mens,Men>Men Clothing>Shorts,13.15
11914618,93234677-a0ae-423c-bef6-a05532f0f24e,2019-08-22 05:54:33.776936 UTC,view_item,f43e7f41-691f-4474-939c-a7487d82ce3c,Crafted Junior Boys 3 Pack Jog Set,Children>Boy's Clothing>Boy's sets,20.14
8511242,222791a6-09c0-497e-8b7a-e05a19e95877,2019-08-08 11:41:39.312886 UTC,view_item,b5861be8-fec3-4058-959d-f47f015f1598,Prince Reflex Mens Tennis Shoes,Men>Men Footwear>Sneakers>Sport sneakers,57.42
6251879,464b63ec-0142-4a72-8cd9-fb0b7f0132d2,2019-07-30 20:04:46.710069 UTC,view_item,fa7b221d-95a1-48a3-96b2-c19a2bc6e7dc,Trendyol Black Patent Leather Detailed Women's...,Women>Women's Footwear>Court shoes,25.26
4502318,46d04df5-6331-4e61-b907-3f29f00b133c,2019-07-22 17:54:34.156284 UTC,view_item,6aa5329b-719e-4087-aaad-8f24a9ce24c0,Slazenger Canvas Kung Snr93,Men>Men Footwear>Sneakers>Low sneakers,16.97
9041043,1ad4cf0a-c156-4888-924d-88d5cf39d788,2019-08-10 14:08:07.641959 UTC,view_item,d5a8b3f3-da4e-4009-89c6-f8102a256a8c,Blowfish Balta Boots,Women>Women's Footwear>Ankle shoes>on wedge heel,38.14


In [25]:
rows_num, cols_num = data.shape
print(rows_num)
print(cols_num)

14803097
7


In [5]:
del events
del purchases

## Atribúty

### customer_id

In [11]:
data.customer_id.describe()

count                                 14803097
unique                                 2211386
top       53f841c7-970b-4e0b-96bf-97a95ecfb734
freq                                      4804
Name: customer_id, dtype: object

In [12]:
len(data[data.customer_id.isna()])

0

In [13]:
rows_num/len(data.customer_id.unique())

6.694035776657716

In [14]:
data.groupby(['customer_id']).count()['timestamp'].median()

2.0

In [14]:
customer_rows = data['customer_id'].value_counts()
customer_rows

53f841c7-970b-4e0b-96bf-97a95ecfb734    4804
9eb76d09-8006-4262-b5eb-459556f338a9    3577
546a1146-64be-4549-a472-a8a4a8ca41b0    3189
a3fe2d55-b8fc-4aaa-8f7f-99f6498b191f    3060
68b2c520-5366-4143-a253-04fd96aeb251    2855
                                        ... 
28f3de37-5d91-451b-9d71-c016ff28765a       1
ddaea6ea-cf1e-4ef3-84d0-4d97f5019dab       1
ee2c4b69-2194-4785-a817-76a30ea8a54b       1
db9f09e8-ea9e-4444-986a-39c973a7dfe3       1
3fe6ea93-4d29-4d6f-990a-6a873f19c2ed       1
Name: customer_id, Length: 2211386, dtype: int64

In [28]:
for i, num in enumerate(customer_rows.value_counts()[:3]):
    print(i+1, num)

1 913643
2 329587
3 199175


In [29]:
(913643)/2211386

0.4131540129131685

In [20]:
(913643+329587)/2211386

0.5621949311427313

In [26]:
for i, num in enumerate(customer_rows.value_counts()[3:50:4]):
    print(4*i+4, num)

4 129721
8 45301
12 22832
16 13311
20 8728
24 6193
28 4481
32 3349
36 2677
40 2124
44 1784
48 1405


- 2,211,386 unikátnych zákazníkov
- žiadne NaN hodnoty
- v priemere vyše 6 záznamov na zákazníka
    - ale median je iba 2
    - 913,643 (41%) zákazníkov má len 1 záznam
    - 56% zákazníkov má 2 záznamy

### timestamp

In [16]:
data.timestamp.describe()

count                           14803097
unique                          14802889
top       2019-07-22 04:33:52.889148 UTC
freq                                   4
Name: timestamp, dtype: object

In [17]:
data.loc[data['timestamp'] == "2019-07-22 04:33:52.889148 UTC"]

,customer_id,timestamp,event_type,product_id,title,category_name,price
4361630,8b57d536-f432-45a9-8b45-3bd54d68ba69,2019-07-22 04:33:52.889148 UTC,view_item,5bbe88b4-9650-4c1a-8f95-9a09c264b2c9,Hot Tuna Crew T Shirt Mens,Men>Men Clothing>T-Shirts & Polo shirts,12.84
4361631,8b57d536-f432-45a9-8b45-3bd54d68ba69,2019-07-22 04:33:52.889148 UTC,view_item,248faf65-d1e2-4d7b-84c8-ce1c459ee086,Lonsdale Jersey Polo Shirt Mens,Men>Men Clothing>T-Shirts & Polo shirts>Polo S...,12.90
4361632,8b57d536-f432-45a9-8b45-3bd54d68ba69,2019-07-22 04:33:52.889148 UTC,view_item,377bdb75-6f46-4c29-b559-9af0c6ba2ae8,Pierre Cardin Web Belt Shorts Mens,Men>Men Clothing>Shorts,17.12
4361633,8b57d536-f432-45a9-8b45-3bd54d68ba69,2019-07-22 04:33:52.889148 UTC,view_item,b2bcef2f-df2d-4c23-aee7-73ca112ef815,Lee Cooper Men's boxers 5-pack,Men>Men Clothing>Underwear>Boxer shorts & trunks,13.10


In [47]:
# data["timestamp"] = pd.to_datetime(data["timestamp"])

In [18]:
min(data.timestamp)

'2019-07-01 00:00:01.101977 UTC'

In [19]:
max(data.timestamp)

'2019-08-31 23:59:59.536067 UTC'

In [7]:
len(data[data.timestamp.isna()])

0

- dátumy z intervalu 2 mesiacov
- žiadne NaN hodnoty
- existujú záznamy typu `view_item`, kde je daný zákazník v danom čase a rôzne produkty. 

### event_type

In [21]:
data.event_type.unique()

array(['view_item', 'purchase_item'], dtype=object)

In [22]:
len(data[data.event_type.isna()])

0

In [23]:
len(data.loc[data['event_type'] == "view_item"])/rows_num

0.9872518568242848

In [24]:
len(data.loc[data['event_type'] == "purchase_item"])/rows_num

0.01274814317571519

- dva typy: `view_item` a `purchase_item`
- pomer je 98.7% ku 1.27%
- žiadne NaN hodnoty

### product_id

In [25]:
data.product_id.describe()

count                                 14803097
unique                                  123837
top       31ae211d-57a9-4bfa-b95f-40776af03c3f
freq                                     52736
Name: product_id, dtype: object

In [60]:
len(data[data.product_id.isna()])

0

In [61]:
rows_num/len(data.product_id.unique())

119.53694776197744

In [62]:
data.groupby(['product_id']).count()['timestamp'].median() 

17.0

In [32]:
product_rows = data['product_id'].value_counts()
product_rows

31ae211d-57a9-4bfa-b95f-40776af03c3f    52736
0f943312-7141-4606-abfa-81fd63a5498f    49141
a8f0292d-5fca-42b9-b0d3-b38e7efa416b    48343
e33cea18-ee3f-4e37-83a7-1d88c54d83b8    45421
608f32cd-210c-4af5-99d8-6182a9678cf5    39801
                                        ...  
6b0e71d0-cdca-4160-8df1-ae7b43ef967c        1
630d92d1-ed2b-4a75-a778-3dacba195eb5        1
0633f706-85d4-4ac6-b34d-7f653fa7139d        1
a3c46419-d8fb-4273-88ba-06dc92b42696        1
326e712f-9733-4761-b458-217f3f695833        1
Name: product_id, Length: 123837, dtype: int64

In [36]:
for i, num in enumerate(product_rows.value_counts()[:50:4]):
    print(4*i+1, num)

1 15982
5 3582
9 2365
13 1792
17 1376
21 1139
25 948
29 810
33 694
37 624
41 528
45 501
49 436


- 123,837 unikátnych produktov
- žiadne NaN hodnoty
- v priemere vyše 119 záznamov na produkt
    - maximum je 52,736
    - minimum je 1
    - median je 17
    - 15,982 produktov má len 1 záznam

### title

In [63]:
data.title.describe()

count                     14801284
unique                       71440
top       Lee Cooper Marl Backpack
freq                        168766
Name: title, dtype: object

In [72]:
data.loc[data['product_id'] == "31ae211d-57a9-4bfa-b95f-40776af03c3f"].title.unique()

array(['Batoh Lee Cooper Marl', 'Lee Cooper Marl Backpack'], dtype=object)

In [64]:
len(data[data.product_id.isna()])

0

- 71,440 unikátnych označení (našli sme 123,837 unikátnych produktov)
    - produkt môže mať viac rôznych označení
- žiadne NaN hodnoty

### price

In [85]:
data.price.describe().apply(lambda x: format(x, 'f'))

count    14758211.000000
mean           33.122065
std           121.532833
min             0.060000
25%            12.710000
50%            21.860000
75%            39.490000
max         32636.460000
Name: price, dtype: object

In [87]:
np.nanmedian(data.price)

21.86

In [73]:
len(data[data.price.isna()])

44886

In [89]:
data.loc[data['product_id'] == "31ae211d-57a9-4bfa-b95f-40776af03c3f"].price.unique()

array([12.9 , 12.73, 13.11, 13.32, 12.49, 12.71, 13.45, 12.65, 12.83,
       11.46, 11.8 , 11.44, 11.38, 12.1 , 11.24, 11.61, 11.99, 11.55,
        9.55,  9.56, 10.61, 10.63, 13.49,  7.32, 12.14, 12.27, 12.33,
       13.04, 12.92, 12.51, 12.11, 12.45, 12.35, 13.09, 16.84, 16.41,
       16.11, 16.03, 16.23, 15.95, 16.  , 15.58, 16.39, 16.97, 13.65,
       14.75, 13.98, 14.02, 14.05, 14.22, 14.38, 14.87, 14.12, 14.37,
       12.46, 12.37, 13.12, 12.04, 13.02, 12.39, 12.68, 12.55, 12.69,
         nan])

- priemer ceny je cca 33
- median ceny je cca 22
- 44,886 NaN hodnôt
- jeden produkt môže mať viac cien

### category_name

In [90]:
data.category_name.describe()

count                                   11367365
unique                                       713
top       Men>Men Footwear>Sneakers>Low sneakers
freq                                      462179
Name: category_name, dtype: object

In [103]:
for pid in data.product_id.sample(10):
    unique_categories = data.loc[data['product_id'] == pid].category_name.unique()
    print(len(unique_categories), unique_categories)

2 [nan 'Men>Men Clothing>T-Shirts & Polo shirts>Polo Shirts']
3 [nan "Women>Women's Clothing>Shirts"
 "Women>Women's Clothing>Shirts>Patterned shirts"]
2 [nan "Women>Women's Clothing>Jackets & Coats>Coats"]
2 [nan "Women>Women's Footwear>Slip-on"]
2 [nan "Women>Women's Clothing>Dresses>Wedding dress"]
2 [nan 'Men>Men Clothing>T-Shirts & Polo shirts>Polo Shirts']
2 [nan "Sports>Running>Running Shoes>Women's Running Shoes"]
2 [nan 'Accessories >Backpacks and Bags>Fashion backpacks']
3 ["Women>Women's Clothing>Pants & Jeans>Pants"
 "Women>Women's Clothing>Pants >Pants"
 "Women>Women's Clothing>Pants >Single-colored pants"]
2 [nan "Women>Women's Clothing>Jumpsuits"]


In [96]:
category_name_nans_num = len(data[data.category_name.isna()])
category_name_nans_num

3435732

In [95]:
category_name_nans_num/rows_num

0.23209548650529008

In [45]:
data["category_name"].str.split(">", expand = True).shape

(14803097, 7)

In [40]:
data_15K = data.sample(15000)

In [116]:
categories = data_15K["category_name"].str.split(">", expand = True)

Keďže `data` má skoro 15M záznamov, spracovanie v niektorkých prípadoch trvá veľmi dlho. Vybrali sme len 15K.

In [123]:
categories.sample(5)

,0,1,2,3,4
2563389,NaN,NaN,NaN,NaN,NaN
7189282,Women,Women's Footwear,Balerinas,None,None
11478593,Men,Men Clothing,T-Shirts & Polo shirts,Short Sleeve,None
6003198,Women,Women's Clothing,Swimmsuits & bikins,One-piece,None
1495798,NaN,None,None,None,None


In [136]:
for i in range(5):
    print('col_{} unique values ='.format(i), len(categories[i].unique()))
    print('col_{} NaN values ='.format(i), len(categories[categories[i].isna()]))
    print()

col_0 unique values = 21
col_0 NaN values = 3455

col_1 unique values = 38
col_1 NaN values = 3551

col_2 unique values = 154
col_2 NaN values = 3669

col_3 unique values = 216
col_3 NaN values = 6845

col_4 unique values = 49
col_4 NaN values = 14513



In [126]:
categories[0].unique()

array([nan, 'Men', 'Women', 'Sports', "Women's Clothing", 'Children',
       'Accessories ', 'Backpacks and Bags', 'Men Clothing',
       "Boy's Clothing", 'Sale - Men', 'Sale - Women', 'Dresses',
       'NEW IN', 'Unlisted', "Children's Footwear", 'strom kategorii',
       'T-Shirts', 'NY FASHION WEEK', 'Sale - Children',
       'Delivery Immediately'], dtype=object)

- **iba** 713 unikátnych kategórií (našli sme 123,837 unikátnych produktov)
- jeden produkt môže mať rôzne `category_name` hodnoty, vrátane NaN
- 23.2% záznamov je NaN

- po rozparsovaní stromu `category_name` vzniklo 7 stĺpcov (t.j. výška stromu)
- smerom od koreňa rastie počet NaN hodnôt a unikátnych hodnôt
- vyskytujú sa tématicky rovnaké len inak napísané kategórie


### Diskusia
Potreba vyriešiť nejednotné atribúty rovnakého produktu:
- `title`
- `category_name`
- možno `cena`

---

# Predspracovanie a oprava dát

In [7]:
categories = data["category_name"].str.split(">", expand = True)
del data["category_name"]
category_names = {}
for i in range(7):
    category_names[i] = "category_{}".format(i) 
categories = categories.rename(columns=category_names)
data = pd.concat([data, categories], axis=1, sort=False)
del categories

Vyparsovanie `category`

In [2]:
# data.to_csv('vi_data/data_train.csv', encoding='utf-8')
data = pd.read_csv('vi_data/data_train.csv', index_col=0)

Uloženie medzi-výsledku

In [219]:
data.sample(3)

,customer_id,timestamp,event_type,product_id,title,price,category_0,category_1,category_2,category_3,category_4,category_5,category_6
13029308,a307441c-2131-4cfe-a3f5-ceba85324cdb,2019-08-26 11:38:54.520998 UTC,view_item,7e40885a-c1e4-48e3-80e3-71951fc25a44,Children's Ski jacket LOAP FALDA,39.52,Children,Boy's Clothing,Jackets,Winter jackets,NaN,NaN,NaN
9634041,0e42ec0e-8d82-4d4a-b6ef-17ef06e3de1a,2019-08-12 19:25:21.494362 UTC,view_item,43bee0d7-2d3f-42ac-bef2-99de2b660be2,Top Secret LADY'S DRESS,15.34,Women,Women's Clothing,Dresses,Mini dress,NaN,NaN,NaN
14697110,7c04bdf3-0ea1-4830-9047-0350d7071ef2,2019-07-31 09:10:17.624419 UTC,purchase_item,ca3ccb9a-933f-45fb-a10a-dfde4eb046ee,Everlast Heavy Bomber Jacket Mens,35.00,Men Clothing,Men,Jackets & Coats,Bomber jackets,NaN,NaN,NaN


## Vytvorenie elastic indexu a naplnenie produktami

**PUT http://localhost:9200/vi_products**
```
{'settings': {'number_of_shards': 1, 'number_of_replicas': 0},
 'mappings': {'properties': {'product_id': {'type': 'keyword'},
   'title': {'type': 'text', 'analyzer': 'english'},
   'price': {'type': 'double'},
   'events_number': {'type': 'integer'},
   'category_0': {'type': 'keyword'},
   'category_1': {'type': 'keyword'},
   'category_2': {'type': 'keyword'},
   'category_3': {'type': 'keyword'},
   'category_4': {'type': 'keyword'},
   'category_5': {'type': 'keyword'},
   'category_6': {'type': 'keyword'}}}}
```

In [194]:
product_ids = list(data.product_id.unique())

In [195]:
len(product_ids)

123837

In [192]:
def fill_index():
    es = Elasticsearch([{'host': '127.0.0.1', 'port': 9200}])
    doc = {}
    for pid in product_ids:
        rows = data[data.product_id == pid]
        doc['product_id'] = pid
        try:
            doc['title'] = list(rows.title.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['price'] = np.mean(rows.price.dropna().unique())
        except IndexError:
            pass
        try:
            doc['events_number'] = len(rows)
        except IndexError:
            pass
        try:
            doc['category_0'] = list(rows.category_0.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['category_1'] = list(rows.category_1.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['category_2'] = list(rows.category_2.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['category_3'] = list(rows.category_3.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['category_4'] = list(rows.category_4.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['category_5'] = list(rows.category_5.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['category_6'] = list(rows.category_6.dropna().unique())[0]
        except IndexError:
            pass
        if np.isnan(doc['price']):
            del doc['price']
        es.index(index='vi_products', body=doc)

In [218]:
# fill_index()

# Obsahový odporúčač

In [365]:
def content_recomendation(product_ids, count=10):
    url = 'http://localhost:9200/vi_products/_search'
    def transform_id(pid):
        data = {'query': {'bool': {'filter': {'term': {'product_id': pid}}}}}
        r = requests.post(url, json=data)
        try:
            return json.loads(r.text)['hits']['hits'][0]['_id']
        except IndexError:
            return None
    
    ids = [transform_id(pid) for pid in product_ids if pid]
    like = [{'_index': 'vi_products', '_id': i} for i in ids if i]
    data = {
        "from" : 0, "size" : count,
        "query": {
            "more_like_this" : {
                "fields" : ["title", "category_0", "category_1", "category_2", "category_3", "category_4", "category_5", "category_6"],
                "like" : like
    }}}    
    print(data)
#     r = requests.post(url, json=data)
#     try: 
#         return [hit['_source']['product_id'] for hit in json.loads(r.text)['hits']['hits']]
#     except KeyError as e:
#         print('KeyError:'+e, r.text) 

# Evaluácia

## Metriky

In [198]:
# def precision_at_k(recommendation, test, k=10):
#     relevance = []
#     for x in recommendation:
#         relevance.append(1 if x in test else 0)
#     k = len(relevance) if len(relevance)<k else k
#     r = np.asarray(relevance)[:k] != 0
#     if r.size != k:
#         raise ValueError('Relevance score length < k')
#     return np.mean(r)

Upravené implementácie z [Ranking Metrics](https://gist.github.com/bwhite/3726239)

In [287]:
def test_precision(customer_id):
    def split_products(products):
        middle = len(products)//2
        return [products[:middle], products[middle:]]
    customer_products = data[data.customer_id == customer_id].sort_values(by ='timestamp')
    if len(customer_products) == 1:
        raise Exception(customer_id + ' has just one product')
    product_ids = list(customer_products.product_id)
    train, test = split_products(product_ids)
    recommendation = content_recomendation(train, count=10)
    return precision_at_k(recommendation, test, k=10)

## Výsledky

In [223]:
customer_ids = list(data.customer_id.unique())

In [309]:
len(customer_ids)

2211386

In [310]:
test_customer_ids = sorted(customer_ids, key=lambda k: random.random())

In [311]:
p = []
number_tested = 100
for cid in test_customer_ids:
    try:
        precision = test_precision(cid)
        if not np.isnan(precision):
            p.append(precision)
    except:
        pass
    if len(p) >= number_tested:
        break
print('number of tested customers', len(p))
print('average precision', np.mean(p))

number of tested customers 100
average precision 0.018000000000000002


# vi-challenge kaggle

In [345]:
vi_challenge_uID = pd.read_csv('vi_challenge_uID.csv', header=None)
vi_challenge_uID.head(5)

,0
0,00077e7e-583f-45a1-8cbc-77f3aca80f45
1,0036fbad-4a20-47f7-a682-aebb275456dc
2,00de3b37-057f-4ade-97f9-3f4f7d869258
3,0107a47a-45bc-4d34-947e-ed6d9c932753
4,011ac83b-0b37-4f9f-9925-9e271e481345


In [372]:
data[data.customer_id == '0036fbad-4a20-47f7-a682-aebb275456dc'].sort_values(by ='timestamp')

,customer_id,timestamp,event_type,product_id,title,price,category_0,category_1,category_2,category_3,category_4,category_5,category_6
4405804,0036fbad-4a20-47f7-a682-aebb275456dc,2019-07-22 09:22:00.231213 UTC,view_item,e5df129d-0477-4408-8ce4-468c6df1954e,Women's dress KILPI DRESS LABATUT-W,29.29,Women,Women's Clothing,Dresses,Casual Dress,NaN,NaN,NaN
4604099,0036fbad-4a20-47f7-a682-aebb275456dc,2019-07-23 08:33:30.238228 UTC,view_item,3abe63bf-07c8-4bcb-817f-f22b46e6ebc5,Full Circle Tankini Top Ladies,11.98,Women,Women's Clothing,Swimmsuits & bikins,Bikini top,NaN,NaN,NaN
4604249,0036fbad-4a20-47f7-a682-aebb275456dc,2019-07-23 08:34:08.595228 UTC,view_item,29d63b9b-c948-4df6-a76a-9ff5e5922003,Full Circle Tankini Top Ladies,13.32,Women,Women's Clothing,Swimmsuits & bikins,NEAKTIVNE NEZARADOVAT Two-pieces,NaN,NaN,NaN
4604583,0036fbad-4a20-47f7-a682-aebb275456dc,2019-07-23 08:35:43.286228 UTC,view_item,24e8d758-f36e-4a46-a3f7-f580c9d8c5e8,Full Circle Tie Skirt Bikini Bottoms Ladies,16.01,Women,Women's Clothing,Swimmsuits & bikins,NaN,NaN,NaN,NaN
4604669,0036fbad-4a20-47f7-a682-aebb275456dc,2019-07-23 08:36:06.496228 UTC,view_item,91a9de35-14c3-4623-b516-24d5fe4c7759,Full Circle Tie Skirt Bikini Bottoms Ladies,16.01,Women,Women's Clothing,Swimmsuits & bikins,NaN,NaN,NaN,NaN
4604705,0036fbad-4a20-47f7-a682-aebb275456dc,2019-07-23 08:36:16.948228 UTC,view_item,24e8d758-f36e-4a46-a3f7-f580c9d8c5e8,Full Circle Tie Skirt Bikini Bottoms Ladies,16.01,Women,Women's Clothing,Swimmsuits & bikins,NaN,NaN,NaN,NaN
4604967,0036fbad-4a20-47f7-a682-aebb275456dc,2019-07-23 08:37:21.155228 UTC,view_item,24e8d758-f36e-4a46-a3f7-f580c9d8c5e8,Full Circle Tie Skirt Bikini Bottoms Ladies,16.01,Women,Women's Clothing,Swimmsuits & bikins,NaN,NaN,NaN,NaN
4604976,0036fbad-4a20-47f7-a682-aebb275456dc,2019-07-23 08:37:22.995228 UTC,view_item,91a9de35-14c3-4623-b516-24d5fe4c7759,Full Circle Tie Skirt Bikini Bottoms Ladies,16.01,Women,Women's Clothing,Swimmsuits & bikins,NaN,NaN,NaN,NaN
4604987,0036fbad-4a20-47f7-a682-aebb275456dc,2019-07-23 08:37:25.110228 UTC,view_item,24e8d758-f36e-4a46-a3f7-f580c9d8c5e8,Full Circle Tie Skirt Bikini Bottoms Ladies,16.01,Women,Women's Clothing,Swimmsuits & bikins,NaN,NaN,NaN,NaN
4605033,0036fbad-4a20-47f7-a682-aebb275456dc,2019-07-23 08:37:41.485228 UTC,view_item,3abe63bf-07c8-4bcb-817f-f22b46e6ebc5,Full Circle Tankini Top Ladies,11.98,Women,Women's Clothing,Swimmsuits & bikins,Bikini top,NaN,NaN,NaN


In [370]:
list(data[data.customer_id == '0036fbad-4a20-47f7-a682-aebb275456dc'].product_id)

['e5df129d-0477-4408-8ce4-468c6df1954e',
 '3abe63bf-07c8-4bcb-817f-f22b46e6ebc5',
 '29d63b9b-c948-4df6-a76a-9ff5e5922003',
 '24e8d758-f36e-4a46-a3f7-f580c9d8c5e8',
 '91a9de35-14c3-4623-b516-24d5fe4c7759',
 '24e8d758-f36e-4a46-a3f7-f580c9d8c5e8',
 '24e8d758-f36e-4a46-a3f7-f580c9d8c5e8',
 '91a9de35-14c3-4623-b516-24d5fe4c7759',
 '24e8d758-f36e-4a46-a3f7-f580c9d8c5e8',
 '3abe63bf-07c8-4bcb-817f-f22b46e6ebc5',
 '3abe63bf-07c8-4bcb-817f-f22b46e6ebc5',
 '23f6a87c-54cb-4ceb-b35f-32ee29903af6',
 '3a246c67-751d-4082-9d2c-903620112a64',
 '3a246c67-751d-4082-9d2c-903620112a64',
 '23f6a87c-54cb-4ceb-b35f-32ee29903af6',
 '24e8d758-f36e-4a46-a3f7-f580c9d8c5e8',
 '3abe63bf-07c8-4bcb-817f-f22b46e6ebc5']

In [371]:
content_recomendation(list(data[data.customer_id == '0036fbad-4a20-47f7-a682-aebb275456dc'].product_id))

{'from': 0, 'size': 10, 'query': {'more_like_this': {'fields': ['title', 'category_0', 'category_1', 'category_2', 'category_3', 'category_4', 'category_5', 'category_6'], 'like': [{'_index': 'vi_products', '_id': 'Iqam3m4BJjk9B7uuzobk'}, {'_index': 'vi_products', '_id': 'MKZZ3m4BJjk9B7uuW3C-'}, {'_index': 'vi_products', '_id': 'DqYR3m4BJjk9B7uu61xc'}, {'_index': 'vi_products', '_id': 'vKZb3m4BJjk9B7uuSnAs'}, {'_index': 'vi_products', '_id': 'wKb33W4BJjk9B7uuw1S4'}, {'_index': 'vi_products', '_id': 'vKZb3m4BJjk9B7uuSnAs'}, {'_index': 'vi_products', '_id': 'vKZb3m4BJjk9B7uuSnAs'}, {'_index': 'vi_products', '_id': 'wKb33W4BJjk9B7uuw1S4'}, {'_index': 'vi_products', '_id': 'vKZb3m4BJjk9B7uuSnAs'}, {'_index': 'vi_products', '_id': 'MKZZ3m4BJjk9B7uuW3C-'}, {'_index': 'vi_products', '_id': 'MKZZ3m4BJjk9B7uuW3C-'}, {'_index': 'vi_products', '_id': 'HKbN3m4BJjk9B7uuWpGy'}, {'_index': 'vi_products', '_id': '3ad65G4BJjk9B7uu9PHV'}, {'_index': 'vi_products', '_id': '3ad65G4BJjk9B7uu9PHV'}, {'_inde

In [361]:
output = list()
for uid in vi_challenge_uID[0][:5]:
    customer_products = list(data[data.customer_id == uid].product_id)
    if customer_products:
        for r in content_recomendation(customer_products):
            output.append([uid, r])
            print([uid, r])
    else:
        for r in list(data.product_id.value_counts().index[:10]):
            output.append([uid, r])
            print([uid, r])

['00077e7e-583f-45a1-8cbc-77f3aca80f45', 'ceb8825f-344b-4872-b4cb-92139ac71446']
['00077e7e-583f-45a1-8cbc-77f3aca80f45', 'da6a5257-2168-4ef8-8e89-37a35ed2b8ab']
['00077e7e-583f-45a1-8cbc-77f3aca80f45', 'f965fa3a-ee44-425d-bc33-737e29cc5685']
['00077e7e-583f-45a1-8cbc-77f3aca80f45', 'a24ad998-d67b-4041-a9b4-40fc37d11b29']
['00077e7e-583f-45a1-8cbc-77f3aca80f45', '6a8e3f66-d282-45d3-806b-c450c56a0124']
['00077e7e-583f-45a1-8cbc-77f3aca80f45', 'a7998640-835d-4a65-8fbb-7cbb632561d1']
['00077e7e-583f-45a1-8cbc-77f3aca80f45', '5d1dced2-4fb3-474b-ad93-28bc3bc40525']
['00077e7e-583f-45a1-8cbc-77f3aca80f45', 'd08bf770-48e2-48ee-a355-f27c410ba4dd']
['00077e7e-583f-45a1-8cbc-77f3aca80f45', '9651f526-fa24-4427-b06b-ab32a07be295']
['00077e7e-583f-45a1-8cbc-77f3aca80f45', '8ea88ef3-9713-476e-9d06-429ea02ae7a5']
['00de3b37-057f-4ade-97f9-3f4f7d869258', '31ae211d-57a9-4bfa-b95f-40776af03c3f']
['00de3b37-057f-4ade-97f9-3f4f7d869258', '0f943312-7141-4606-abfa-81fd63a5498f']
['00de3b37-057f-4ade-97f9-3f

In [358]:
submission = pd.DataFrame(output, columns=['customer_id', 'product_id'])
submission.to_csv('harnusekSubmission.csv', index=False)
len(submission)

40

## Otázky
* Odporucat aj produkty, ktore uz boli kupene/pozrete?
* Z ktorej tabulky vychadzat?

## Zapracovat
* boostovanie v elastiku
* dalsie metriky
* vyber trenovacich produktov

---

In [283]:
test_customer_ids[:5]

['b35cb1e5-df23-478a-abda-4a81105c3f08',
 '99af6bb5-4b87-4d4b-84c5-db5d6f62f975',
 'b60c5b74-e1d2-4704-a0c4-82bee5c24169',
 'cb42bb4d-e8f3-40fe-89ef-32e1b4808c9e',
 'ae74c624-1346-47db-a97f-f35036731a2d']

In [255]:
list(x.product_id)

def split_products(products):
    middle = len(products)//2
    return [products[:middle], products[middle:]]

In [302]:
x = data[data.customer_id == '91271cd2-8230-49d8-ae78-d4034e6e86ce']
x

,customer_id,timestamp,event_type,product_id,title,price,category_0,category_1,category_2,category_3,category_4,category_5,category_6
81220,91271cd2-8230-49d8-ae78-d4034e6e86ce,2019-07-01 11:49:43.19673 UTC,view_item,98c60a5c-5255-49ff-8a80-25532301e084,Lee Cooper dámske šľapky,15.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81355,91271cd2-8230-49d8-ae78-d4034e6e86ce,2019-07-01 11:50:22.79273 UTC,view_item,98c60a5c-5255-49ff-8a80-25532301e084,Lee Cooper dámske šľapky,15.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [303]:
train, test = split_products(list(x.product_id))

In [304]:
train

['98c60a5c-5255-49ff-8a80-25532301e084']

In [267]:
content_recomendation(train)

['fb773131-8ba4-4f41-b894-c6a952a6b101',
 '46a7e225-cc99-40b5-a810-224032d2bfd1',
 'cc55fbd5-a7e7-4bbf-945e-96d60fc32ac5',
 'ae5d59a7-0ae2-4622-82c4-9a35b8614699',
 '023912ff-dc38-4c4d-b22e-77c7162f8a5f',
 '387ee3e2-0b20-4a3d-ab79-b0d908f58d73',
 '295944ef-9b68-4f4f-a120-cad1eb5bc62c',
 '9267ac37-b0ec-4105-9dc8-168536030bb5',
 'e603ca39-1c46-4e94-80ad-98982eec34be',
 'd115ce65-869d-4682-b499-2b8541625b7a']

In [305]:
test

['98c60a5c-5255-49ff-8a80-25532301e084']

In [301]:
test_precision('91271cd2-8230-49d8-ae78-d4034e6e86ce')

nan